## Capstone 3 - Customer Churn 

### Problem: 

#### Churn rate is a marketing metric that describes the number of customers who leave a business over a specific time period 30 days. Every user is assigned a prediction value that estimates their state of churn at any given time. This value is based on:

##### User demographic information
##### Browsing behavior
##### Historical purchase data among other information

In [81]:
### Question: Are you selling to your ideal customer profile? (ICP)
### How can we keep the bottom <2% of customers?

In [60]:
import pandas as pd
import matplotlib.pyplot as plt

#### Loading file

In [23]:
df = pd.read_csv('churn.csv')

In [24]:
df.head()

,Unnamed: 0,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,...,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback,churn_risk_score
0,0,18,F,XW0DQ7H,Village,Platinum Membership,2017-08-17,No,xxxxxxxx,Gift Vouchers/Coupons,...,300.63,53005.25,17.0,781.75,Yes,Yes,No,Not Applicable,Products always in Stock,0
1,1,32,F,5K0N3X1,City,Premium Membership,2017-08-28,?,CID21329,Gift Vouchers/Coupons,...,306.34,12838.38,10.0,NaN,Yes,No,Yes,Solved,Quality Customer Care,0
2,2,44,F,1F2TCL3,Town,No Membership,2016-11-11,Yes,CID12313,Gift Vouchers/Coupons,...,516.16,21027.00,22.0,500.69,No,Yes,Yes,Solved in Follow-up,Poor Website,1
3,3,37,M,VJGJ33N,City,No Membership,2016-10-29,Yes,CID3793,Gift Vouchers/Coupons,...,53.27,25239.56,6.0,567.66,No,Yes,Yes,Unsolved,Poor Website,1
4,4,31,F,SVZXCWB,City,No Membership,2017-09-12,No,xxxxxxxx,Credit/Debit Card Offers,...,113.13,24483.66,16.0,663.06,No,Yes,Yes,Solved,Poor Website,1


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36992 entries, 0 to 36991
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Unnamed: 0                    36992 non-null  int64  
 1   age                           36992 non-null  int64  
 2   gender                        36992 non-null  object 
 3   security_no                   36992 non-null  object 
 4   region_category               31564 non-null  object 
 5   membership_category           36992 non-null  object 
 6   joining_date                  36992 non-null  object 
 7   joined_through_referral       36992 non-null  object 
 8   referral_id                   36992 non-null  object 
 9   preferred_offer_types         36704 non-null  object 
 10  medium_of_operation           36992 non-null  object 
 11  internet_option               36992 non-null  object 
 12  last_visit_time               36992 non-null  object 
 13  d

#### Column(churn_risk_score) factors in our unique and proprietary predictions of how long a user will remain a customer. This score is updated every day for all users who have a minimum of one conversion. The values assigned are between 1 and 5.

In [26]:
df['churn_risk_score'].unique()

array([0, 1])

In [27]:
df.isnull().sum()

Unnamed: 0                         0
age                                0
gender                             0
security_no                        0
region_category                 5428
membership_category                0
joining_date                       0
joined_through_referral            0
referral_id                        0
preferred_offer_types            288
medium_of_operation                0
internet_option                    0
last_visit_time                    0
days_since_last_login              0
avg_time_spent                     0
avg_transaction_value              0
avg_frequency_login_days           0
points_in_wallet                3443
used_special_discount              0
offer_application_preference       0
past_complaint                     0
complaint_status                   0
feedback                           0
churn_risk_score                   0
dtype: int64

In [28]:
df['region_category'].unique() #Unknown fill 
#Inference model to the class (guess)
#Remove the churn_risk_score 
#Time column becomes where the data started

array(['Village', 'City', 'Town', nan], dtype=object)

In [29]:
df['region_category'].value_counts()

Town       14128
City       12737
Village     4699
Name: region_category, dtype: int64

In [47]:
df['region_category'] = df['region_category'].fillna('Unknown')

In [30]:
df['preferred_offer_types'].unique()

array(['Gift Vouchers/Coupons', 'Credit/Debit Card Offers',
       'Without Offers', nan], dtype=object)

In [49]:
df['preferred_offer_types'] = df['preferred_offer_types'].fillna('Unknown')

In [36]:
df['points_in_wallet'].unique()

array([781.75      ,          nan, 500.69      , ..., 242.97962545,
       639.51      , 197.26441364])

In [50]:
#NAN can be no points on file 
df['points_in_wallet'] = df['points_in_wallet'].fillna(0)

In [51]:
df.isnull().sum()

Unnamed: 0                      0
age                             0
gender                          0
security_no                     0
region_category                 0
membership_category             0
joining_date                    0
joined_through_referral         0
referral_id                     0
preferred_offer_types           0
medium_of_operation             0
internet_option                 0
last_visit_time                 0
days_since_last_login           0
avg_time_spent                  0
avg_transaction_value           0
avg_frequency_login_days        0
points_in_wallet                0
used_special_discount           0
offer_application_preference    0
past_complaint                  0
complaint_status                0
feedback                        0
churn_risk_score                0
dtype: int64

#### Cleaning Data

In [52]:
df2 = df.drop(columns =['churn_risk_score', 'Unnamed: 0'])

In [53]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36992 entries, 0 to 36991
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   age                           36992 non-null  int64  
 1   gender                        36992 non-null  object 
 2   security_no                   36992 non-null  object 
 3   region_category               36992 non-null  object 
 4   membership_category           36992 non-null  object 
 5   joining_date                  36992 non-null  object 
 6   joined_through_referral       36992 non-null  object 
 7   referral_id                   36992 non-null  object 
 8   preferred_offer_types         36992 non-null  object 
 9   medium_of_operation           36992 non-null  object 
 10  internet_option               36992 non-null  object 
 11  last_visit_time               36992 non-null  object 
 12  days_since_last_login         36992 non-null  int64  
 13  a

In [54]:
df2.head()

,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,medium_of_operation,...,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback
0,18,F,XW0DQ7H,Village,Platinum Membership,2017-08-17,No,xxxxxxxx,Gift Vouchers/Coupons,?,...,17,300.63,53005.25,17.0,781.75,Yes,Yes,No,Not Applicable,Products always in Stock
1,32,F,5K0N3X1,City,Premium Membership,2017-08-28,?,CID21329,Gift Vouchers/Coupons,Desktop,...,16,306.34,12838.38,10.0,0.00,Yes,No,Yes,Solved,Quality Customer Care
2,44,F,1F2TCL3,Town,No Membership,2016-11-11,Yes,CID12313,Gift Vouchers/Coupons,Desktop,...,14,516.16,21027.00,22.0,500.69,No,Yes,Yes,Solved in Follow-up,Poor Website
3,37,M,VJGJ33N,City,No Membership,2016-10-29,Yes,CID3793,Gift Vouchers/Coupons,Desktop,...,11,53.27,25239.56,6.0,567.66,No,Yes,Yes,Unsolved,Poor Website
4,31,F,SVZXCWB,City,No Membership,2017-09-12,No,xxxxxxxx,Credit/Debit Card Offers,Smartphone,...,20,113.13,24483.66,16.0,663.06,No,Yes,Yes,Solved,Poor Website


In [ ]:
#Feedback is an important column 
#Past_complaint 
#Complaint_status

In [55]:
df2['joining_date'] = pd.to_datetime(df2['joining_date'])

In [57]:
df2['joining_date'] 

0       2017-08-17
1       2017-08-28
2       2016-11-11
3       2016-10-29
4       2017-09-12
           ...    
36987   2017-09-21
36988   2016-06-27
36989   2016-09-11
36990   2017-06-15
36991   2015-10-23
Name: joining_date, Length: 36992, dtype: datetime64[ns]

#### Dropping Calculated Values 

In [73]:
df2[['avg_frequency_login_days','avg_transaction_value','avg_time_spent']]

,avg_frequency_login_days,avg_transaction_value,avg_time_spent
0,17.0,53005.25,300.630000
1,10.0,12838.38,306.340000
2,22.0,21027.00,516.160000
3,6.0,25239.56,53.270000
4,16.0,24483.66,113.130000
...,...,...,...
36987,6.0,27277.68,-650.682759
36988,28.0,11069.71,-638.123421
36989,Error,38127.56,154.940000
36990,20.0,2378.86,482.610000


In [76]:
df3 = df2.drop(columns= ['avg_frequency_login_days','avg_transaction_value','avg_time_spent'])

In [77]:
#Days since last login 
df3['days_since_last_login'].sort_values(ascending=False)

14701     26
12845     26
13953     26
16648     26
28690     26
        ... 
34919   -999
34922   -999
34924   -999
24164   -999
24804   -999
Name: days_since_last_login, Length: 36992, dtype: int64

In [78]:
#How many have been 26 days
df3.sort_values('days_since_last_login', ascending=False)

,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,preferred_offer_types,medium_of_operation,internet_option,last_visit_time,days_since_last_login,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback
14701,10,M,N4F4SQ1,City,Premium Membership,2016-02-17,Yes,xxxxxxxx,Without Offers,Desktop,Wi-Fi,02:12:51,26,769.060000,Yes,No,No,Not Applicable,Poor Website
12845,49,F,I957KN3,City,Gold Membership,2016-12-02,?,CID23980,Without Offers,Smartphone,Fiber_Optic,01:57:17,26,672.700000,No,Yes,No,Not Applicable,Poor Product Quality
13953,24,M,CGXUMWW,Town,No Membership,2015-03-25,No,xxxxxxxx,Without Offers,Smartphone,Fiber_Optic,11:30:09,26,586.490000,No,Yes,Yes,Solved in Follow-up,No reason specified
16648,40,M,HG1C1UF,Village,Silver Membership,2017-08-02,Yes,CID34703,Credit/Debit Card Offers,Smartphone,Mobile_Data,00:36:18,26,607.950000,No,Yes,No,Not Applicable,Poor Product Quality
28690,17,F,W2UXOR2,Unknown,Platinum Membership,2015-06-06,Yes,CID36682,Credit/Debit Card Offers,Desktop,Mobile_Data,11:45:17,26,749.690000,Yes,No,Yes,Unsolved,Poor Customer Service
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34919,19,F,576IMVG,Town,Gold Membership,2016-10-31,Yes,CID14737,Gift Vouchers/Coupons,Desktop,Mobile_Data,11:41:20,-999,1007.176202,Yes,No,No,Not Applicable,No reason specified
34922,25,M,CZ2PENL,Village,Premium Membership,2017-06-29,No,xxxxxxxx,Gift Vouchers/Coupons,Desktop,Fiber_Optic,18:08:52,-999,770.260000,Yes,No,No,Not Applicable,Quality Customer Care
34924,62,F,2RIIC5I,Town,Basic Membership,2016-05-09,Yes,CID2769,Without Offers,Smartphone,Mobile_Data,18:00:44,-999,645.830000,No,Yes,Yes,Solved in Follow-up,Poor Customer Service
24164,55,F,MAVHXFN,Town,Silver Membership,2016-07-21,Yes,CID14900,Credit/Debit Card Offers,Smartphone,Mobile_Data,14:36:46,-999,0.000000,No,Yes,No,Not Applicable,Poor Customer Service


In [79]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36992 entries, 0 to 36991
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   age                           36992 non-null  int64         
 1   gender                        36992 non-null  object        
 2   security_no                   36992 non-null  object        
 3   region_category               36992 non-null  object        
 4   membership_category           36992 non-null  object        
 5   joining_date                  36992 non-null  datetime64[ns]
 6   joined_through_referral       36992 non-null  object        
 7   referral_id                   36992 non-null  object        
 8   preferred_offer_types         36992 non-null  object        
 9   medium_of_operation           36992 non-null  object        
 10  internet_option               36992 non-null  object        
 11  last_visit_time             

In [80]:
#Extract my dataset as a csv 
df3.to_csv('Clean_Customer_Churn.csv')